# DELFIN Dashboard
Generate CONTROL files, submit jobs, manage queue, and build ORCA inputs - all in one place.

In [ ]:
# === Imports and Setup ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import py3Dmol
import os
import subprocess
import shutil
import re
import sys
import numpy as np
from pathlib import Path

# RDKit for SMILES conversion
from rdkit import Chem
from rdkit.Chem import AllChem

# stk for metal complexes (optional)
try:
    import stk
    HAS_STK = True
except ImportError:
    HAS_STK = False

# Ensure local DELFIN is importable
_deflin_root = None
for _base in [Path.cwd(), *Path.cwd().parents]:
    if (_base / 'software' / 'delfin' / 'delfin').is_dir():
        _deflin_root = _base / 'software' / 'delfin'
        break
    if (_base / 'delfin').is_dir():
        _deflin_root = _base
        break
if _deflin_root and str(_deflin_root) not in sys.path:
    sys.path.insert(0, str(_deflin_root))

import importlib
import delfin.config
import delfin.common.control_validator
importlib.reload(delfin.common.control_validator)
importlib.reload(delfin.config)
from delfin.config import validate_control_text
from delfin.common.control_validator import ORCA_FUNCTIONALS, ORCA_BASIS_SETS, DISP_CORR_VALUES, _RI_JKX_KEYWORDS

# === Path Setup ===
HOME = Path.home()
try:
    import IPython
    NOTEBOOK_DIR = Path(IPython.extract_module_locals()[1]['__vsc_ipynb_file__']).parent
except:
    NOTEBOOK_DIR = Path.cwd()

def find_root_dir(start_dir: Path):
    cur = start_dir
    while True:
        if (cur / 'software' / 'delfin').is_dir():
            return cur
        if cur == cur.parent:
            return None
        cur = cur.parent

ROOT_DIR = find_root_dir(NOTEBOOK_DIR)
if ROOT_DIR is None:
    ROOT_DIR = HOME

BASE_DIR = ROOT_DIR / 'calc'

_submit_candidates = [
    ROOT_DIR / 'software' / 'delfin' / 'examples' / 'example_Job_Submission_Scripts' / 'BwUniCluster' / 'submit_sh',
    NOTEBOOK_DIR / 'submit_sh',
]
SUBMIT_TEMPLATES_DIR = None
for _path in _submit_candidates:
    if _path.exists():
        SUBMIT_TEMPLATES_DIR = _path
        break
if SUBMIT_TEMPLATES_DIR is None:
    SUBMIT_TEMPLATES_DIR = _submit_candidates[0]

_goat_candidates = [
    ROOT_DIR / 'software' / 'CONTROL_Templates' / 'only_GOAT.txt',
    ROOT_DIR / 'CONTROL_Templates' / 'only_GOAT.txt',
    NOTEBOOK_DIR / 'CONTROL_Templates' / 'only_GOAT.txt',
]
ONLY_GOAT_TEMPLATE_PATH = None
for _path in _goat_candidates:
    if _path.exists():
        ONLY_GOAT_TEMPLATE_PATH = _path
        break
if ONLY_GOAT_TEMPLATE_PATH is None:
    ONLY_GOAT_TEMPLATE_PATH = _goat_candidates[0]

JOB_TYPE_TEMPLATES = {
    'Short (24h)': 'submit_short.sh',
    'Standard (48h)': 'submit_standard.sh',
    'Long (72h)': 'submit_long.sh',
    'Extra Long (120h)': 'submit_extralong.sh'
}

print(f'ROOT_DIR: {ROOT_DIR}')
print(f'BASE_DIR: {BASE_DIR}')
print('Setup complete.')

In [ ]:
# === Helper Functions ===

def contains_metal(smiles):
    metals = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Be', 'Mg', 'Ca', 'Sr', 'Ba',
              'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
              'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
              'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy',
              'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os',
              'Ir', 'Pt', 'Au', 'Hg', 'Al', 'Ga', 'In', 'Tl', 'Sn', 'Pb',
              'Bi', 'Po', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu']
    for metal in metals:
        if re.search(rf'\[{metal}[+\-\d\]@]', smiles, re.IGNORECASE):
            return True
        if re.search(rf'\[{metal}\]', smiles, re.IGNORECASE):
            return True
    return False

def mol_from_smiles_rdkit(smiles, allow_metal=False):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return mol, None
        if not allow_metal:
            return None, 'Invalid SMILES'
        mol = Chem.MolFromSmiles(smiles, sanitize=False)
        if mol is None:
            return None, 'Invalid SMILES (without sanitizing)'
        try:
            Chem.SanitizeMol(mol, sanitizeOps=(
                Chem.SanitizeFlags.SANITIZE_ALL
                ^ Chem.SanitizeFlags.SANITIZE_PROPERTIES
                ^ Chem.SanitizeFlags.SANITIZE_KEKULIZE
            ))
            return mol, 'partial sanitizing'
        except Exception:
            return mol, 'sanitizing skipped'
    except Exception as e:
        return None, f'RDKit error: {e}'

def smiles_to_xyz(smiles):
    has_metal = contains_metal(smiles)
    stk_error = None
    if has_metal and HAS_STK:
        try:
            bb = stk.BuildingBlock(smiles)
            mol = bb.to_rdkit_mol()
            if mol.GetNumConformers() == 0:
                AllChem.EmbedMolecule(mol, randomSeed=42, useRandomCoords=True)
            method = 'stk'
        except Exception as e:
            stk_error = str(e)
            mol = None
            method = None
    else:
        mol = None
        method = None
    if mol is None:
        mol, rdkit_note = mol_from_smiles_rdkit(smiles, allow_metal=has_metal)
        if mol is None:
            if stk_error:
                return None, 0, None, f'RDKit: {rdkit_note}; stk: {stk_error}'
            return None, 0, None, rdkit_note
        try:
            mol = Chem.AddHs(mol)
        except Exception:
            pass
        try:
            params = AllChem.ETKDGv3()
            params.randomSeed = 42
            params.useRandomCoords = True
            try:
                result = AllChem.EmbedMolecule(mol, params, maxAttempts=50)
            except TypeError:
                result = AllChem.EmbedMolecule(mol, params)
            if result == -1:
                params2 = AllChem.ETKDGv3()
                params2.randomSeed = 42
                params2.useRandomCoords = True
                try:
                    result = AllChem.EmbedMolecule(mol, params2, maxAttempts=200)
                except TypeError:
                    result = AllChem.EmbedMolecule(mol, params2)
                if result == -1:
                    return None, 0, None, 'Could not generate 3D structure'
            method = 'RDKit'
            if rdkit_note:
                method = f'RDKit ({rdkit_note})'
        except Exception as e:
            return None, 0, None, f'RDKit error: {str(e)}'
    try:
        conf = mol.GetConformer()
        num_atoms = mol.GetNumAtoms()
        xyz_lines = []
        for i in range(num_atoms):
            atom = mol.GetAtomWithIdx(i)
            pos = conf.GetAtomPosition(i)
            xyz_lines.append(f'{atom.GetSymbol():2s} {pos.x:12.6f} {pos.y:12.6f} {pos.z:12.6f}')
        return '\n'.join(xyz_lines), num_atoms, method, None
    except Exception as e:
        return None, 0, None, f'Coordinate error: {str(e)}'

def is_smiles(text):
    text = text.strip()
    if '\n' in text:
        return False
    return bool(re.match(r'^[A-Za-z0-9\[\]()=#@+\-\\/.:,%]+$', text))

def clean_input_data(input_text):
    text = input_text.strip()
    if not text:
        return '', 'empty'
    if is_smiles(text):
        return text, 'smiles'
    lines = text.split('\n')
    if len(lines) < 2:
        return text, 'xyz'
    first_line = lines[0].strip()
    try:
        atom_count = int(first_line)
        cleaned_lines = lines[2:]
        return '\n'.join(cleaned_lines).strip(), 'xyz'
    except ValueError:
        return text, 'xyz'

def parse_resource_settings(control_text):
    pal_match = re.search(r'^\s*PAL\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    maxcore_match = re.search(r'^\s*maxcore\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    pal = int(pal_match.group(1)) if pal_match else None
    maxcore = int(maxcore_match.group(1)) if maxcore_match else None
    return pal, maxcore

def apply_resources_to_submit_script(script_text, pal, maxcore):
    if pal:
        script_text = re.sub(r'(?m)^#SBATCH\s+--ntasks=\d+.*$', f'#SBATCH --ntasks={pal}', script_text)
    if pal and maxcore:
        mem_mb = pal * maxcore
        script_text = re.sub(r'(?m)^#SBATCH\s+--mem=\S+.*$', f'#SBATCH --mem={mem_mb}M', script_text)
    return script_text

print('Helper functions loaded.')

In [ ]:
# === Main Tabbed UI ===

DEFAULT_CONTROL = """input_file=input.txt
NAME=
SMILES=
charge=[CHARGE]
------------------------------------
Solvation:
implicit_solvation_model=CPCM
solvent=[SOLVENT]
XTB_SOLVATOR=no
number_explicit_solv_molecules=2
------------------------------------
Global geometry optimisation:
xTB_method=XTB2
XTB_OPT=no
XTB_GOAT=no
CREST=no
multiplicity_global_opt=
------------------------------------
IMAG=yes
IMAG_scope=initial
IMAG_option=2
allow_imaginary_freq=0
IMAG_sp_energy_window=1e-3
IMAG_optimize_candidates=no
------------------------------------
calc_prop_of_interest=no
properties_of_interest=IP,EA
------------------------------------
Redox steps:
calc_initial=yes
oxidation_steps=1,2,3
reduction_steps=1,2,3
method=classic|manually|OCCUPIER
calc_potential_method=2
------------------------------------
ESD module (excited state dynamics):
ESD_modul=no
ESD_modus=TDDFT|deltaSCF|hybrid1
ESD_frequency=yes
states=S1,T1,S2,T2
ISCs=S1>T1,T1>S1
ICs=S2>S1
emission_rates=f,p
phosp_IROOT=1,2,3
phosp_keywords=
fluor_keywords=
TROOTSSL=-1,0,1
addition_S0=
DOHT=TRUE
ESD_LINES=LORENTZ
ESD_LINEW=50
ESD_INLINEW=250
ESD_NPOINTS=131072
ESD_MAXTIME=12000
hybrid1_geom_MaxIter=60
--------------------
Electrical Properties:
elprop_Dipole=no
elprop_Quadrupole=no
elprop_Hyperpol=no
elprop_Polar=no
elprop_PolarVelocity=no
elprop_PolarDipQuad=no
elprop_PolarQuadQuad=no
--------------------
deltaSCF Settings:
deltaSCF_DOMOM=true
deltaSCF_PMOM=true
deltaSCF_keepinitialref=true
deltaSCF_SOSCFHESSUP=LSR1
deltaSCF_keywords=FreezeAndRelease
deltaSCF_maxiter=300
deltaSCF_SOSCFConvFactor=500
deltaSCF_SOSCFMaxStep=0.1
--------------------
TDDFT Settings:
TDDFT_TDDFT_maxiter=500
TDDFT_nroots=15
TDDFT_maxdim=30
TDDFT_TDA=FALSE
TDDFT_followiroot=true
TDDFT_SOC=false
------------------------------------
MANUALLY:
multiplicity_0=
additions_0=
additions_TDDFT=
additions_T1=
additions_S1=
multiplicity_ox1=
additions_ox1=
multiplicity_ox2=
additions_ox2=
multiplicity_ox3=
additions_ox3=
multiplicity_red1=
additions_red1=
multiplicity_red2=
additions_red2=
multiplicity_red3=
additions_red3=
------------------------------------
Level of Theory:
functional=PBE0
disp_corr=D4
ri_jkx=RIJCOSX
relativity=ZORA
aux_jk=def2/J
aux_jk_rel=SARC/J
main_basisset=def2-SVP
main_basisset_rel=ZORA-def2-SVP
metal_basisset=def2-TZVP
metal_basisset_rel=SARC-ZORA-TZVP
first_coordination_sphere_metal_basisset=no
first_coordination_sphere_scale=1.3
geom_opt=OPT
freq_type=FREQ
initial_guess=PModel
temperature=298.15
maxiter=125
qmmm_option=QM/PBEH-3c
------------------------------------
Reference value:
E_ref=
------------------------------------
Literature_reference=
reference_CV=V Vs. Fc+/Fc
E_00_exp=
E_red_exp=
E_red_2_exp=
E_red_3_exp=
E_ox_exp=
E_ox_2_exp=
E_ox_3_exp=
*E_red_exp=
*E_ox_exp=
------------------------------------
Prints:
print_MOs=yes
print_Loewdin_population_analysis=no
------------------------------------
Resource Settings:
PAL=40
maxcore=6000
parallel_workflows=yes
pal_jobs=4
orca_parallel_strategy=auto
enable_job_timeouts=no
job_timeout_hours=36
opt_timeout_hours=14
frequency_timeout_hours=36
sp_timeout_hours=3
------------------------------------
Automatic Error Recovery & Retry:
enable_auto_recovery=yes
max_recovery_attempts=1
enable_adaptive_parallelism=yes
enable_performance_metrics=yes
------------------------------------
OCCUPIER-Settings:
--------------------
OCCUPIER_method=auto
OCCUPIER_tree=own
OWN_TREE_PURE_WINDOW=3
OWN_progressive_from=no
fob_equal_weights=yes
frequency_calculation_OCCUPIER=no
occupier_selection=tolerance
occupier_precision=3
occupier_epsilon=5e-4
clean_override_window_h=0.002
clean_quality_improvement=0.05
clean_quality_good=0.05
maxiter_occupier=125
geom_opt_OCCUPIER=OPT
pass_wavefunction=no
approximate_spin_projection_APMethod=2
"""

common_layout = widgets.Layout(width='500px')
common_style = {'description_width': 'initial'}
_converted_xyz_cache = {'smiles': None, 'xyz': None}

# =============================================
# TAB 1: Submit Job
# =============================================

job_name_widget = widgets.Text(value='', placeholder='e.g. Fe_Complex_Ox', description='Job Name:', layout=common_layout, style=common_style)
job_type_widget = widgets.RadioButtons(options=['Short (24h)', 'Standard (48h)', 'Long (72h)', 'Extra Long (120h)'], value='Standard (48h)', description='Job Type:', style=common_style, layout=widgets.Layout(width='500px'))
coords_widget = widgets.Textarea(value='', placeholder='Paste XYZ or SMILES', description='Input:', layout=widgets.Layout(width='500px', height='180px'), style=common_style)
convert_smiles_button = widgets.Button(description='CONVERT SMILES', button_style='info', layout=widgets.Layout(width='180px', height='35px'))
smiles_batch_widget = widgets.Textarea(value='', placeholder='Name;SMILES;key=value', description='SMILES List:', layout=widgets.Layout(width='500px', height='120px'), style=common_style)
submit_smiles_list_button = widgets.Button(description='SUBMIT SMILES LIST', button_style='success', layout=widgets.Layout(width='220px', height='40px'))
smiles_batch_output = widgets.Output()
control_widget = widgets.Textarea(value=DEFAULT_CONTROL, description='CONTROL.txt:', layout=widgets.Layout(width='500px', height='400px'), style=common_style)
validate_button = widgets.Button(description='VALIDATE', button_style='warning', layout=widgets.Layout(width='150px', height='50px'))
submit_button = widgets.Button(description='SUBMIT JOB', button_style='primary', layout=widgets.Layout(width='150px', height='50px'))
output_area = widgets.Output()
validate_output = widgets.Output()
mol_output = widgets.Output(layout=widgets.Layout(border='2px solid #1976d2', width='100%', height='400px', overflow='hidden'))

# Only GOAT section
only_goat_charge = widgets.IntText(value=0, description='Charge:', style=common_style, layout=widgets.Layout(width='140px'))
only_goat_solvent = widgets.Text(value='', placeholder='e.g. water', description='Solvent:', style=common_style, layout=widgets.Layout(width='200px'))
only_goat_submit_button = widgets.Button(description='SUBMIT ONLY GOAT', button_style='success', layout=widgets.Layout(width='180px', height='40px'))
only_goat_output = widgets.Output()

def update_molecule_view(change=None):
    global _converted_xyz_cache
    with mol_output:
        clear_output()
        raw_input = coords_widget.value.strip()
        if not raw_input:
            print('Enter XYZ coordinates or SMILES.')
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            return
        cleaned_data, input_type = clean_input_data(raw_input)
        if input_type == 'smiles':
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            print('SMILES detected. Click CONVERT SMILES.')
            return
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        coords = cleaned_data
        lines = [l for l in coords.split('\n') if l.strip()]
        num_atoms = len(lines)
        xyz_data = f'{num_atoms}\nGenerated\n{coords}'
        mol_view = py3Dmol.view(width=580, height=380)
        mol_view.addModel(xyz_data, 'xyz')
        mol_view.setStyle({}, {'stick': {'radius': 0.15}, 'sphere': {'scale': 0.22}})
        mol_view.zoomTo()
        mol_view.show()

coords_widget.observe(update_molecule_view, names='value')

def handle_convert_smiles(button):
    global _converted_xyz_cache
    raw_input = coords_widget.value.strip()
    if not raw_input:
        with mol_output:
            clear_output()
            print('Enter SMILES first.')
        return
    with mol_output:
        clear_output()
        print('Converting...')
    cleaned_data, input_type = clean_input_data(raw_input)
    if input_type != 'smiles':
        with mol_output:
            clear_output()
            print('Not a SMILES string.')
        return
    xyz_string, num_atoms, method, error = smiles_to_xyz(cleaned_data)
    if error:
        with mol_output:
            clear_output()
            print(f'Error: {error}')
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        return
    _converted_xyz_cache = {'smiles': cleaned_data, 'xyz': xyz_string}
    coords_widget.value = f'{num_atoms}\nConverted ({method})\n{xyz_string}'

convert_smiles_button.on_click(handle_convert_smiles)

def reset_form():
    global _converted_xyz_cache
    job_name_widget.value = ''
    coords_widget.value = ''
    smiles_batch_widget.value = ''
    control_widget.value = DEFAULT_CONTROL
    job_type_widget.value = 'Standard (48h)'
    only_goat_charge.value = 0
    only_goat_solvent.value = ''
    _converted_xyz_cache = {'smiles': None, 'xyz': None}
    with mol_output:
        clear_output()
        print('Enter XYZ coordinates or SMILES.')

def handle_validate_control(button):
    with validate_output:
        clear_output()
        errors = validate_control_text(control_widget.value)
        if errors:
            print('Validation failed:')
            for e in errors:
                print(f'- {e}')
        else:
            print('CONTROL.txt is valid.')

validate_button.on_click(handle_validate_control)

def handle_submit(button):
    with output_area:
        clear_output()
        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        control_content = control_widget.value
        raw_input = coords_widget.value.strip()
        if not job_name:
            print('Error: Job name required!')
            return
        if not raw_input:
            print('Error: Input required!')
            return
        errors = validate_control_text(control_content)
        if errors:
            print('CONTROL.txt validation failed:')
            for e in errors:
                print(f'- {e}')
            return
        input_content, input_type = clean_input_data(raw_input)
        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
            input_type = 'xyz (from SMILES)'
        if not input_content:
            print('Error: No valid input!')
            return
        safe_job_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print('Error: Invalid job name!')
            return
        job_dir = BASE_DIR / safe_job_name
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename
        if not template_path.exists():
            print(f'Error: Template not found: {template_path}')
            return
        try:
            job_dir.mkdir(parents=True, exist_ok=True)
            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace('SMILES=', f"SMILES={_converted_xyz_cache['smiles']}")
            (job_dir / 'CONTROL.txt').write_text(control_content)
            (job_dir / 'input.txt').write_text(input_content)
            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            pal, maxcore = parse_resource_settings(control_content)
            submit_script = apply_resources_to_submit_script(submit_script, pal, maxcore)
            submit_path = job_dir / 'submit_job.sh'
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)
            result = subprocess.run(['sbatch', 'submit_job.sh'], cwd=job_dir, capture_output=True, text=True)
            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print(f'Job submitted! ID: {job_id}')
                print(f'Directory: {job_dir}')
                reset_form()
            else:
                print(f'Error: {result.stderr}')
        except Exception as e:
            print(f'Error: {str(e)}')

submit_button.on_click(handle_submit)

def handle_submit_smiles_list(button):
    with smiles_batch_output:
        clear_output()
        job_prefix = job_name_widget.value.strip()
        if not job_prefix:
            print('Error: Job name required!')
            return
        control_content_base = control_widget.value
        errors = validate_control_text(control_content_base)
        if errors:
            print('CONTROL.txt validation failed')
            return
        job_type = job_type_widget.value
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename
        if not template_path.exists():
            print(f'Template not found: {template_path}')
            return
        entries = [l.strip() for l in smiles_batch_widget.value.splitlines() if l.strip()]
        if not entries:
            print('SMILES list is empty.')
            return
        for idx, entry in enumerate(entries, 1):
            if ';' not in entry:
                print(f'Line {idx}: Missing delimiter')
                continue
            parts = [p.strip() for p in entry.split(';') if p.strip()]
            if len(parts) < 2:
                print(f'Line {idx}: Missing name or SMILES')
                continue
            name_raw, smiles = parts[0], parts[1]
            extras = {}
            for part in parts[2:]:
                if '=' in part:
                    k, v = part.split('=', 1)
                    extras[k.strip()] = v.strip()
            safe_name = ''.join(c for c in name_raw if c.isalnum() or c in ('_', '-'))
            job_name = f'{job_prefix}_{safe_name}'
            safe_job_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
            xyz_string, num_atoms, method, error = smiles_to_xyz(smiles)
            if error:
                print(f'Line {idx}: {error}')
                continue
            job_dir = BASE_DIR / safe_job_name
            job_dir.mkdir(parents=True, exist_ok=True)
            control_content = re.sub(r'(?m)^SMILES=.*$', f'SMILES={smiles}', control_content_base)
            for key, value in extras.items():
                pattern = rf'(?m)^{re.escape(key)}\s*=.*$'
                if re.search(pattern, control_content):
                    control_content = re.sub(pattern, f'{key}={value}', control_content)
                else:
                    control_content = control_content.rstrip() + f'\n{key}={value}\n'
            (job_dir / 'CONTROL.txt').write_text(control_content)
            (job_dir / 'input.txt').write_text(xyz_string)
            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            pal, maxcore = parse_resource_settings(control_content)
            submit_script = apply_resources_to_submit_script(submit_script, pal, maxcore)
            submit_path = job_dir / 'submit_job.sh'
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)
            result = subprocess.run(['sbatch', 'submit_job.sh'], cwd=job_dir, capture_output=True, text=True)
            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '?'
                print(f'Submitted {safe_job_name} (ID: {job_id})')
            else:
                print(f'Failed {safe_job_name}')

submit_smiles_list_button.on_click(handle_submit_smiles_list)

def handle_only_goat_submit(button):
    with only_goat_output:
        clear_output()
        job_name = job_name_widget.value.strip()
        if not job_name:
            print('Error: Job name required!')
            return
        raw_input = coords_widget.value.strip()
        if not raw_input:
            print('Error: Input required!')
            return
        solvent_value = only_goat_solvent.value.strip()
        if not solvent_value:
            print('Error: Solvent required!')
            return
        input_content, input_type = clean_input_data(raw_input)
        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
        if not input_content:
            print('Error: No valid input!')
            return
        safe_job_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not ONLY_GOAT_TEMPLATE_PATH or not ONLY_GOAT_TEMPLATE_PATH.exists():
            print('Error: GOAT template not found')
            return
        job_dir = BASE_DIR / safe_job_name
        template_filename = JOB_TYPE_TEMPLATES.get(job_type_widget.value, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename
        if not template_path.exists():
            print(f'Template not found: {template_path}')
            return
        try:
            job_dir.mkdir(parents=True, exist_ok=True)
            control_template = ONLY_GOAT_TEMPLATE_PATH.read_text()
            control_content = control_template.replace('[CHARGE]', str(only_goat_charge.value)).replace('[SOLVENT]', solvent_value)
            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace('SMILES=', f"SMILES={_converted_xyz_cache['smiles']}")
            (job_dir / 'CONTROL.txt').write_text(control_content)
            (job_dir / 'input.txt').write_text(input_content)
            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            pal, maxcore = parse_resource_settings(control_content)
            submit_script = apply_resources_to_submit_script(submit_script, pal, maxcore)
            submit_path = job_dir / 'submit_job.sh'
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)
            result = subprocess.run(['sbatch', 'submit_job.sh'], cwd=job_dir, capture_output=True, text=True)
            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print(f'GOAT job submitted! ID: {job_id}')
                reset_form()
            else:
                print(f'Error: {result.stderr}')
        except Exception as e:
            print(f'Error: {str(e)}')

only_goat_submit_button.on_click(handle_only_goat_submit)

spacer = widgets.Label(value='', layout=widgets.Layout(height='10px'))
spacer_large = widgets.Label(value='', layout=widgets.Layout(height='20px'))

submit_left = widgets.VBox([
    job_name_widget, spacer, job_type_widget, spacer_large,
    widgets.HTML('<b>Input (XYZ or SMILES):</b>'), coords_widget, spacer, convert_smiles_button,
    spacer_large, widgets.HTML('<b>Batch SMILES:</b>'), smiles_batch_widget, spacer, submit_smiles_list_button, smiles_batch_output,
    spacer_large, widgets.HTML('<b>CONTROL.txt:</b>'), control_widget,
    widgets.HBox([validate_button, submit_button]), output_area, validate_output
], layout=widgets.Layout(width='50%', padding='10px'))

submit_right = widgets.VBox([
    widgets.HTML('<b>Molecule Preview:</b>'), mol_output,
    spacer_large, widgets.HTML('<b>Only GOAT:</b>'),
    widgets.HBox([only_goat_charge, only_goat_solvent, only_goat_submit_button]),
    only_goat_output
], layout=widgets.Layout(width='50%', padding='10px'))

tab1_content = widgets.HBox([submit_left, submit_right])

# =============================================
# TAB 2: Recalc
# =============================================

recalc_folder_dropdown = widgets.Dropdown(options=[], description='Job Folder:', layout=widgets.Layout(width='500px'), style=common_style)
recalc_control_widget = widgets.Textarea(value='', description='CONTROL.txt:', layout=widgets.Layout(width='100%', height='500px'), style=common_style)
recalc_button = widgets.Button(description='SUBMIT RECALC', button_style='warning', layout=widgets.Layout(width='200px', height='40px'))
recalc_output = widgets.Output()

def refresh_recalc_folders():
    if not BASE_DIR.exists():
        recalc_folder_dropdown.options = []
        return
    folders = sorted([p.name for p in BASE_DIR.iterdir() if p.is_dir()])
    recalc_folder_dropdown.options = folders
    recalc_folder_dropdown.value = folders[0] if folders else None

def load_recalc_control(change=None):
    with recalc_output:
        clear_output()
    folder = recalc_folder_dropdown.value
    if not folder:
        recalc_control_widget.value = ''
        return
    control_path = BASE_DIR / folder / 'CONTROL.txt'
    if control_path.exists():
        recalc_control_widget.value = control_path.read_text()
    else:
        recalc_control_widget.value = ''
        with recalc_output:
            print('CONTROL.txt not found')

recalc_folder_dropdown.observe(load_recalc_control, names='value')

def _patch_submit_for_recalc(script_text):
    lines = script_text.splitlines(True)
    for i, line in enumerate(lines):
        m = re.match(r'^(\s*)delfin(\s+.*)?(\s*)$', line)
        if not m:
            continue
        indent, args, trail = m.group(1), (m.group(2) or ''), m.group(3)
        if '--recalc' in args:
            return script_text, True
        lines[i] = f'{indent}delfin --recalc{args}{trail}'
        return ''.join(lines), True
    return script_text, False

def handle_recalc(button):
    with recalc_output:
        clear_output()
        folder = recalc_folder_dropdown.value
        if not folder:
            print('Select a job folder.')
            return
        job_dir = BASE_DIR / folder
        if not job_dir.exists():
            print('Folder does not exist.')
            return
        errors = validate_control_text(recalc_control_widget.value)
        if errors:
            print('Validation failed:')
            for e in errors:
                print(f'- {e}')
            return
        (job_dir / 'CONTROL.txt').write_text(recalc_control_widget.value)
        submit_path = job_dir / 'submit_job.sh'
        if not submit_path.exists():
            print('submit_job.sh not found')
            return
        submit_content = submit_path.read_text()
        pal, maxcore = parse_resource_settings(recalc_control_widget.value)
        submit_content = apply_resources_to_submit_script(submit_content, pal, maxcore)
        new_content, replaced = _patch_submit_for_recalc(submit_content)
        if not replaced:
            print('Could not patch submit script')
            return
        submit_path.write_text(new_content)
        result = subprocess.run(['sbatch', 'submit_job.sh'], cwd=job_dir, capture_output=True, text=True)
        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '?'
            print(f'Recalc submitted! ID: {job_id}')
        else:
            print(f'Error: {result.stderr}')

recalc_button.on_click(handle_recalc)
refresh_recalc_folders()

recalc_refresh_btn = widgets.Button(description='REFRESH FOLDERS', button_style='info', layout=widgets.Layout(width='150px', height='35px'))
recalc_refresh_btn.on_click(lambda b: refresh_recalc_folders())

tab2_content = widgets.VBox([
    widgets.HTML('<h3>Recalc DELFIN Job</h3>'),
    widgets.HTML('<p>Select a job folder, edit CONTROL.txt, and resubmit.</p>'),
    widgets.HBox([recalc_folder_dropdown, recalc_refresh_btn]),
    recalc_control_widget, spacer,
    recalc_button, recalc_output
], layout=widgets.Layout(padding='10px'))

# =============================================
# TAB 3: Job Status
# =============================================

_job_data = []
job_table_html = widgets.HTML(value='<i>Loading...</i>')
job_start_html = widgets.HTML(value='')
job_dropdown = widgets.Dropdown(options=[], description='Select Job:', layout=widgets.Layout(width='400px'), style={'description_width': '80px'})
job_status_output = widgets.Output()

def refresh_job_list(button=None):
    global _job_data
    with job_status_output:
        clear_output()
    try:
        result = subprocess.run(['squeue', '-u', os.environ.get('USER', ''), '-o', '%.12i %.12P %.35j %.10u %.3t %.12M %.6D %R'], capture_output=True, text=True, timeout=10)
        start_result = subprocess.run(['squeue', '-u', os.environ.get('USER', ''), '--start', '-o', '%.12i %.35j %.20S'], capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            lines = result.stdout.strip().split('\n')
            if len(lines) < 2:
                job_table_html.value = '<p><i>No jobs in queue.</i></p>'
                job_start_html.value = ''
                job_dropdown.options = []
                _job_data = []
                return
            job_lines = lines[1:]
            _job_data = []
            dropdown_options = []
            for line in job_lines:
                if not line.strip():
                    continue
                parts = line.split()
                if len(parts) >= 1:
                    job_id = parts[0].strip()
                    job_name = parts[2].strip() if len(parts) > 2 else 'unknown'
                    _job_data.append({'id': job_id, 'line': line, 'name': job_name})
                    dropdown_options.append((f'{job_id} - {job_name}', job_id))
            table_html = '''<style>.job-table{font-family:monospace;font-size:12px;border-collapse:collapse;width:100%}.job-table th,.job-table td{padding:6px 10px;text-align:left;border-bottom:1px solid #ddd}.job-table th{background-color:#2196F3;color:white}.job-table tr:hover{background-color:#f5f5f5}</style><table class="job-table"><tr><th>JOBID</th><th>PARTITION</th><th>NAME</th><th>USER</th><th>ST</th><th>TIME</th><th>NODES</th><th>NODELIST/REASON</th></tr>'''
            for job in _job_data:
                parts = job['line'].split(None, 7)
                while len(parts) < 8:
                    parts.append('')
                table_html += f'<tr><td><b>{parts[0]}</b></td><td>{parts[1]}</td><td>{parts[2]}</td><td>{parts[3]}</td><td>{parts[4]}</td><td>{parts[5]}</td><td>{parts[6]}</td><td>{parts[7]}</td></tr>'
            table_html += '</table>'
            job_table_html.value = table_html
            if start_result.returncode == 0:
                start_lines = start_result.stdout.strip().split('\n')
                if len(start_lines) > 1:
                    pending_jobs = []
                    for line in start_lines[1:]:
                        if line.strip() and 'N/A' not in line:
                            parts = line.split(None, 2)
                            if len(parts) >= 3:
                                pending_jobs.append({'id': parts[0].strip(), 'name': parts[1].strip(), 'start': parts[2].strip()})
                    if pending_jobs:
                        start_html = '<div style="margin-top:10px;padding:10px;background-color:#fff3cd;border:1px solid #ffc107;border-radius:4px;"><b>Estimated Start Times:</b><br><table style="font-family:monospace;font-size:12px;margin-top:5px;">'
                        for pj in pending_jobs:
                            start_html += f"<tr><td><b>{pj['id']}</b></td><td style='padding-left:15px;'>{pj['name']}</td><td style='padding-left:15px;'>{pj['start']}</td></tr>"
                        start_html += '</table></div>'
                        job_start_html.value = start_html
                    else:
                        job_start_html.value = ''
                else:
                    job_start_html.value = ''
            else:
                job_start_html.value = ''
            if dropdown_options:
                job_dropdown.options = dropdown_options
                job_dropdown.value = dropdown_options[0][1]
            else:
                job_dropdown.options = []
            with job_status_output:
                clear_output()
                print(f'{len(_job_data)} job(s) found')
        else:
            with job_status_output:
                print(f'Error: {result.stderr}')
    except Exception as e:
        with job_status_output:
            print(f'Error: {e}')

def cancel_selected_job(button):
    with job_status_output:
        clear_output()
        if not job_dropdown.value:
            print('No job selected.')
            return
        job_id = job_dropdown.value
        result = subprocess.run(['scancel', str(job_id)], capture_output=True, text=True)
        if result.returncode == 0:
            print(f'Job {job_id} cancelled.')
            refresh_job_list()
        else:
            print(f'Error: {result.stderr or result.stdout}')

refresh_button = widgets.Button(description='REFRESH', button_style='info', layout=widgets.Layout(width='120px', height='40px'))
cancel_button = widgets.Button(description='CANCEL JOB', button_style='danger', layout=widgets.Layout(width='150px', height='40px'))
refresh_button.on_click(refresh_job_list)
cancel_button.on_click(cancel_selected_job)
refresh_job_list()

tab3_content = widgets.VBox([
    widgets.HTML('<h3>Job Status</h3>'),
    widgets.HTML('<p>Select a job and click CANCEL to cancel it.</p>'),
    job_table_html, job_start_html,
    widgets.HBox([job_dropdown, cancel_button, refresh_button], layout=widgets.Layout(margin='10px 0')),
    job_status_output
], layout=widgets.Layout(padding='10px'))

# =============================================
# TAB 4: ORCA Builder
# =============================================

orca_method_options = sorted(ORCA_FUNCTIONALS)
orca_basis_options = sorted(ORCA_BASIS_SETS)
orca_dispersion_options = ['None'] + sorted(v for v in DISP_CORR_VALUES if v)
orca_ri_options = ['None'] + sorted(_RI_JKX_KEYWORDS)

orca_job_name = widgets.Text(value='', placeholder='e.g. water_opt', description='Job Name:', layout=widgets.Layout(width='350px'), style={'description_width': '100px'})
orca_coords = widgets.Textarea(value='', placeholder='Paste XYZ coordinates', description='Coordinates:', layout=widgets.Layout(width='100%', height='300px'), style={'description_width': '100px'})
orca_charge = widgets.IntText(value=0, description='Charge:', layout=widgets.Layout(width='150px'), style={'description_width': '100px'})
orca_multiplicity = widgets.IntText(value=1, description='Mult:', layout=widgets.Layout(width='150px'), style={'description_width': '100px'})
orca_method = widgets.Dropdown(options=orca_method_options, value='PBE0', description='Method:', layout=widgets.Layout(width='200px'), style={'description_width': '80px'})
orca_job_type = widgets.Dropdown(options=['SP', 'OPT', 'FREQ', 'OPT FREQ'], value='OPT', description='Job Type:', layout=widgets.Layout(width='200px'), style={'description_width': '80px'})
orca_basis = widgets.Dropdown(options=orca_basis_options, value='def2-SVP', description='Basis:', layout=widgets.Layout(width='200px'), style={'description_width': '80px'})
orca_dispersion = widgets.Dropdown(options=orca_dispersion_options, value='D4', description='Disp:', layout=widgets.Layout(width='200px'), style={'description_width': '80px'})
orca_ri = widgets.Dropdown(options=orca_ri_options, value='RIJCOSX', description='RI:', layout=widgets.Layout(width='200px'), style={'description_width': '80px'})
orca_aux_basis = widgets.Dropdown(options=['None', 'def2/J', 'def2/JK'], value='def2/J', description='Aux:', layout=widgets.Layout(width='200px'), style={'description_width': '80px'})
orca_cpcm_enabled = widgets.Checkbox(value=False, description='CPCM', layout=widgets.Layout(width='80px'))
orca_solvent = widgets.Dropdown(options=['water', 'acetonitrile', 'dmso', 'dmf', 'methanol', 'ethanol', 'thf', 'dichloromethane', 'chloroform', 'toluene', 'hexane'], value='water', description='Solvent:', layout=widgets.Layout(width='180px'), style={'description_width': '60px'})
orca_pal = widgets.IntText(value=40, description='PAL:', layout=widgets.Layout(width='150px'), style={'description_width': '80px'})
orca_maxcore = widgets.IntText(value=6000, description='MaxCore:', layout=widgets.Layout(width='150px'), style={'description_width': '80px'})
orca_slurm_time = widgets.Text(value='12:00:00', description='Time:', layout=widgets.Layout(width='200px'), style={'description_width': '80px'})
orca_additional = widgets.Text(value='', placeholder='Additional keywords', description='Extra:', layout=widgets.Layout(width='350px'), style={'description_width': '80px'})
orca_preview = widgets.Textarea(value='', description='Preview:', layout=widgets.Layout(width='100%', height='400px'), style={'description_width': '80px'})
orca_submit_btn = widgets.Button(description='SUBMIT ORCA JOB', button_style='success', layout=widgets.Layout(width='180px', height='40px'))
orca_output = widgets.Output()

def generate_orca_input():
    keywords = [orca_method.value, orca_job_type.value, orca_basis.value]
    if orca_dispersion.value != 'None':
        keywords.append(orca_dispersion.value)
    if orca_ri.value != 'None':
        keywords.append(orca_ri.value)
        keywords.append(orca_aux_basis.value)
    if orca_cpcm_enabled.value:
        keywords.append(f'CPCM({orca_solvent.value})')
    if orca_additional.value.strip():
        keywords.append(orca_additional.value.strip())
    return f'''! {' '.join(keywords)}

%pal
  nprocs {orca_pal.value}
end

%maxcore {orca_maxcore.value}

* xyz {orca_charge.value} {orca_multiplicity.value}
{orca_coords.value.strip()}
*
'''

def update_orca_preview(change=None):
    orca_preview.value = generate_orca_input()

for w in [orca_method, orca_job_type, orca_basis, orca_dispersion, orca_ri, orca_aux_basis, orca_charge, orca_multiplicity, orca_pal, orca_maxcore, orca_coords, orca_additional, orca_cpcm_enabled, orca_solvent]:
    w.observe(update_orca_preview, names='value')
update_orca_preview()

def _patch_submit_for_orca(script_text, job_name):
    lines = script_text.splitlines(True)
    for i, line in enumerate(lines):
        m = re.match(r'^(\s*)delfin(\s+.*)?(\s*)$', line)
        if not m:
            continue
        indent, trail = m.group(1), m.group(3)
        lines[i] = f'{indent}delfin run_orca --input {job_name}.inp --output {job_name}.out{trail}'
        return ''.join(lines), True
    return script_text, False

def handle_orca_submit(button):
    with orca_output:
        clear_output()
        job_name = orca_job_name.value.strip()
        if not job_name:
            print('Job name required!')
            return
        if not orca_coords.value.strip():
            print('Coordinates required!')
            return
        safe_job_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        job_dir = BASE_DIR / safe_job_name
        job_dir.mkdir(parents=True, exist_ok=True)
        inp_content = orca_preview.value.strip() if orca_preview.value.strip() else generate_orca_input()
        (job_dir / f'{safe_job_name}.inp').write_text(inp_content)
        template_path = SUBMIT_TEMPLATES_DIR / 'submit_standard.sh'
        if not template_path.exists():
            print('Template not found')
            return
        template_content = template_path.read_text()
        submit_script = template_content.replace('{job_name}', safe_job_name)
        total_mem = orca_pal.value * orca_maxcore.value
        submit_script = re.sub(r'(?m)^#SBATCH\s+--ntasks=\d+.*$', f'#SBATCH --ntasks={orca_pal.value}', submit_script)
        submit_script = re.sub(r'(?m)^#SBATCH\s+--mem=\S+.*$', f'#SBATCH --mem={total_mem}M', submit_script)
        submit_script = re.sub(r'(?m)^#SBATCH\s+--time=\S+.*$', f'#SBATCH --time={orca_slurm_time.value}', submit_script)
        submit_script, replaced = _patch_submit_for_orca(submit_script, safe_job_name)
        if not replaced:
            print('Could not patch submit script')
            return
        submit_path = job_dir / 'submit_orca.sh'
        submit_path.write_text(submit_script)
        submit_path.chmod(0o755)
        result = subprocess.run(['sbatch', 'submit_orca.sh'], cwd=job_dir, capture_output=True, text=True)
        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '?'
            print(f'ORCA job submitted! ID: {job_id}')
            print(f'Directory: {job_dir}')
        else:
            print(f'Error: {result.stderr}')

orca_submit_btn.on_click(handle_orca_submit)

orca_left = widgets.VBox([
    orca_job_name, orca_coords,
    widgets.HBox([orca_charge, orca_multiplicity]),
    widgets.HBox([orca_method, orca_job_type]),
    widgets.HBox([orca_basis, orca_dispersion]),
    widgets.HBox([orca_ri, orca_aux_basis]),
    widgets.HBox([orca_cpcm_enabled, orca_solvent]),
    widgets.HBox([orca_pal, orca_maxcore]),
    orca_slurm_time, orca_additional,
    orca_submit_btn, orca_output
], layout=widgets.Layout(width='45%', padding='10px'))

orca_right = widgets.VBox([
    widgets.HTML('<b>ORCA Input Preview (editable):</b>'),
    orca_preview
], layout=widgets.Layout(width='55%', padding='10px'))

tab4_content = widgets.VBox([
    widgets.HTML('<h3>ORCA Input Builder</h3>'),
    widgets.HBox([orca_left, orca_right])
], layout=widgets.Layout(padding='10px'))

# =============================================
# Create Tabs
# =============================================

tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content, tab4_content])
tabs.set_title(0, 'Submit Job')
tabs.set_title(1, 'Recalc')
tabs.set_title(2, 'Job Status')
tabs.set_title(3, 'ORCA Builder')

display(widgets.HTML('<h2 style="color:#1976d2;">DELFIN Dashboard</h2>'))
display(tabs)